# Face Recognition utilizando Haar Cascade

In [1]:
import cv2
import sys
import numpy
import os 

In [2]:
haar_file = 'haarcascade_frontalface_default.xml'
datasets = 'FotosPruebaHaar'
sub_data = 'Federico'  # Nombre de quien se le hara la prueba

In [3]:
path = os.path.join(datasets, sub_data) 
if not os.path.isdir(path): 
    os.mkdir(path) 

(width, height) = (130, 100) 
face_cascade = cv2.CascadeClassifier(haar_file)  

### Toma de fotos para entrenar el modelo

In [4]:
webcam = cv2.VideoCapture(0) 
count = 1
while count < 30:  
    (_, im) = webcam.read() 
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY) 
    faces = face_cascade.detectMultiScale(gray, 1.3, 4) 
    for (x, y, w, h) in faces: 
        cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 0), 2) 
        face = gray[y:y + h, x:x + w] 
        face_resize = cv2.resize(face, (width, height)) 
        cv2.imwrite('% s/% s.png' % (path, count), face_resize) 
    count += 1
      
    cv2.imshow('OpenCV', im) 
    key = cv2.waitKey(10) 
    if key == 27: 
        break

### Ejecución del modelo

In [5]:
# Create a list of images and a list of corresponding names 
(images, labels, names, id) = ([], [], {}, 0) 
for (subdirs, dirs, files) in os.walk(datasets): 
    for subdir in dirs: 
        names[id] = subdir 
        subjectpath = os.path.join(datasets, subdir) 
        for filename in os.listdir(subjectpath): 
            path = subjectpath + '/' + filename 
            label = id
            images.append(cv2.imread(path, 0)) 
            labels.append(int(label)) 
        id += 1
(width, height) = (130, 100) 

(images, labels) = [numpy.array(lis) for lis in [images, labels]] 

In [6]:
model = cv2.face.LBPHFaceRecognizer_create() 
model.train(images, labels) 

In [7]:
webcam = cv2.VideoCapture(0) 
while True: 
    (_, im) = webcam.read() 
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY) 
    faces = face_cascade.detectMultiScale(gray, 1.3, 5) 
    for (x, y, w, h) in faces: 
        cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 0), 2) 
        face = gray[y:y + h, x:x + w] 
        face_resize = cv2.resize(face, (width, height)) 
        # Try to recognize the face 
        prediction = model.predict(face_resize) 
        cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 3) 
  
        if prediction[1]<500: 
  
           cv2.putText(im, '% s - %.0f' % 
(names[prediction[0]], prediction[1]), (x-10, y-10),  
cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0)) 
        else: 
          cv2.putText(im, 'not recognized',  
(x-10, y-10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0)) 
  
    cv2.imshow('OpenCV', im) 
      
    key = cv2.waitKey(10) 
    if key == 27: 
        break

KeyboardInterrupt: 